<a href="https://colab.research.google.com/github/ayush-dhanker/Image-Classification-on-MNIST/blob/main/model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from time import perf_counter
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as py

In [14]:
# normalizing and transforming
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

# Loading Datasets
train_dataset=datasets.MNIST(root='./data',train=True,download=True,transform=transform)
test_dataset=datasets.MNIST(root='./data',train=False,download=True,transform=transform)

# Data loaders
train_loader=DataLoader(train_dataset,batch_size=128,shuffle=True, drop_last=True, num_workers=2)
test_loader=DataLoader(test_dataset,batch_size=128,shuffle=False, num_workers=2)

print(int(len(train_loader.dataset)/train_loader.batch_size))

468


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# building model
model= nn.Sequential(
      nn.Flatten(),
      nn.Linear(784,512),
      nn.ReLU(),
      nn.Linear(512,128),
      nn.ReLU(),
      nn.Linear(128,10)
).to(device)

print("model = ",model)
with torch.no_grad():
    print("Maximum weight before custom init: ", model[1].weight.max())


def glorot_init(layer: nn.Module):
    if isinstance(layer, nn.Linear):
        nn.init.xavier_uniform_(layer.weight)
        nn.init.zeros_(layer.bias)


with torch.no_grad():
    model.apply(glorot_init)
    print("Maximum weight after custom init", model[1].weight.max())

Using cpu device
model =  Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=128, bias=True)
  (4): ReLU()
  (5): Linear(in_features=128, out_features=10, bias=True)
)
Maximum weight before custom init:  tensor(0.0357)
Maximum weight after custom init tensor(0.0680)


In [4]:
loss_fn=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(), lr=0.03)

In [31]:
# train_model
def train_model(
    model: nn.Module,
    loss_fn: nn.Module,
    optimizer: optim.Optimizer,
    training_loader: DataLoader,
    validation_loader: DataLoader,
    n_epochs:int,
    verbose:bool=True
    ):



  train_len=len(training_loader.dataset)
  steps_per_epoch = train_len//training_loader.batch_size

  print("Running {} epochs at {} steps per epoch ".format(n_epochs,steps_per_epoch))

  train_acc=[]
  train_loss=[]
  val_acc=[]
  val_loss=[]

  for epoch in range(n_epochs):
    if verbose:
            print("Starting epoch {}...".format(epoch + 1), end=" ")

    start_time = perf_counter()
    epoch_train_loss=[]
    epoch_train_acc=[]

    model.train()
    for batch_idx,(input_batch,label_batch) in enumerate(training_loader):
      batch_loss,batch_accuracy = training_set(input_batch,label_batch,model,loss_fn,optimizer)
      epoch_train_loss.append(batch_loss.item())
      epoch_train_acc.append(batch_accuracy.item())

    end_time = perf_counter()
    time_taken = end_time - start_time

    # evaluating
    validation_loss, val_accuracy = evaluate(model,validation_loader,loss_fn)

    val_acc.append(val_accuracy.item())
    val_loss.append(validation_loss.item())
    train_acc.append(np.mean(epoch_train_acc))
    train_loss.append(np.mean(epoch_train_loss))

    if verbose:
            print("Time taken: {} seconds".format(time_taken))
            print("\tTrain/val loss: {} / {}".format(train_loss[-1], val_loss[-1]))
            print("\tTrain/val accuracy: {} / {}".format(train_acc[-1], val_acc[-1]))

  return {"train_loss": np.array(train_loss), "train_acc": np.array(train_acc),
            "val_loss": np.array(val_loss), "val_acc": np.array(val_loss)}


def training_set(
      input: torch.tensor,
      label: torch.tensor,
      model: nn.Module,
      loss_fn: nn.Module,
      optimizer: optim.Optimizer):

    input=input.to(device)
    label=label.to(device)
    output_batch=model(input)
    loss_batch=loss_fn(output_batch, label)

    loss_batch.backward()
    optimizer.step()
    optimizer.zero_grad()

    with torch.no_grad():
      batch_acc=accuracy(label,output_batch)
    # see loss_batch.item()
    return loss_batch, batch_acc


def evaluate(
      model:nn.Module,
      dataloader:DataLoader,
      loss_fn:nn.Module):
    model.eval()
    size=len(dataloader.dataset)
    num_batches=len(dataloader)
    loss,correct=0,0

    with torch.no_grad():
      for input,label in dataloader:
        input=input.to(device)
        label=label.to(device)
        prediction=model(input)
        loss+=loss_fn(prediction,label)
        correct += (prediction.argmax(axis=1)==label).type(torch.float).sum()

    loss/=num_batches
    val_accuracy=correct/size
    return loss, val_accuracy

def accuracy(labels:torch.tensor,
               outputs:torch.tensor)->torch.tensor:
               predictions=torch.argmax(outputs,axis=-1)
               matches=labels==predictions
               return matches.float().mean()

In [32]:
metrics = train_model(model, loss_fn, optimizer, train_loader, test_loader, n_epochs=25)

Running 25 epochs at 468 steps per epoch 
Starting epoch 1... Time taken: 14.004115915999591 seconds
	Train/val loss: 0.008577954465360181 / 0.06210579350590706
	Train/val accuracy: 0.9993656517094017 / 0.9807000160217285
Starting epoch 2... Time taken: 13.907658698999967 seconds
	Train/val loss: 0.00771361013996797 / 0.06166772544384003
	Train/val accuracy: 0.9995993589743589 / 0.9812999963760376
Starting epoch 3... Time taken: 14.138782378000087 seconds
	Train/val loss: 0.007134221703894675 / 0.062419354915618896
	Train/val accuracy: 0.9996494391025641 / 0.9817000031471252
Starting epoch 4... Time taken: 14.417706552999789 seconds
	Train/val loss: 0.00658716739460818 / 0.06260804831981659
	Train/val accuracy: 0.999732905982906 / 0.9811000227928162
Starting epoch 5... Time taken: 13.9489416790002 seconds
	Train/val loss: 0.006131560179259644 / 0.06264001131057739
	Train/val accuracy: 0.9997662927350427 / 0.98089998960495
Starting epoch 6... Time taken: 13.859612996999658 seconds
	Trai